In [4]:
import pandas as pd
import os
import re
import numpy as np


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
folder_path = '/content/drive/MyDrive/ColabNotebooks/Datasets/QBC8_AI_project_data_Q2'
json_files = [file for file in os.listdir(folder_path) if file.endswith('.json')]
apple_json = pd.read_json(folder_path + '/Apple_data.json')

In [7]:
mobile_dataframes = []
for json_file in json_files:
    file_path = os.path.join(folder_path, json_file)
    try:
        df = pd.read_json(file_path)
        # ----------------------------
        all_data_list = []
        for brand_key, brand_value in df.items():
            for phone_key, phone_value in brand_value.items():
                # Extracting parent feature groups
                try:
                    network_info = phone_value.get("Network", {})
                except AttributeError:
                    continue  # Skip entries without details (specific to Sony data)
                launch_info = phone_value.get("Launch", {})
                body_info = phone_value.get("Body", {})
                display_info = phone_value.get("Display", {})
                platform_info = phone_value.get("Platform", {})
                memory_info = phone_value.get("Memory", {})
                battery_info = phone_value.get("Battery", {})
                misc_info = phone_value.get("Misc", {})
                comms_info = phone_value.get("Comms", {})
                sound_info = phone_value.get("Sound", {})
                features_info = phone_value.get("Features", {})

                # Initialize dictionaries for extracted features
                network_info_dict = {}
                launch_info_dict = {}
                body_info_dict = {}
                display_info_dict = {}
                platform_info_dict = {}
                memory_info_dict = {}
                battery_info_dict = {}
                misc_info_dict = {}
                comms_info_dict = {}
                sound_info_dict = {}
                features_info_dict = {}

                # Combine dictionaries from JSON arrays
                for dictionary in network_info:
                    network_info_dict.update(dictionary)
                for dictionary in launch_info:
                    launch_info_dict.update(dictionary)
                for i, dictionary in enumerate(memory_info):
                    if 'SIM' in dictionary:
                        if i == 1:  # First "SIM"
                            memory_info_dict['SIM'] = dictionary.get('SIM', None)
                    else:
                        memory_info_dict.update(dictionary)
                for dictionary in body_info:
                    body_info_dict.update(dictionary)
                for dictionary in display_info:
                    display_info_dict.update(dictionary)
                for dictionary in platform_info:
                    platform_info_dict.update(dictionary)
                for dictionary in battery_info:
                    battery_info_dict.update(dictionary)
                for dictionary in misc_info:
                    misc_info_dict.update(dictionary)
                for dictionary in comms_info:
                    comms_info_dict.update(dictionary)
                for dictionary in sound_info:
                    sound_info_dict.update(dictionary)
                for i, dictionary in enumerate(features_info):
                    if 'Sensors' in dictionary:
                        if i == 1:  # First "Sensors"
                            features_info_dict['Sensors'] = dictionary.get('Sensors', None)
                        elif i == 2:  # Second "Sensors"
                            features_info_dict['-'] = dictionary.get('Sensors', None)
                    else:
                        features_info_dict.update(dictionary)
                for i, dictionary in enumerate(memory_info):
                    if 'Internal' in dictionary:
                        if i == 1:  # First "Internal"
                            memory_info_dict['t_Internal_capacity'] = dictionary.get('Internal', None)
                        elif i == 2:  # Second "Internal"
                            memory_info_dict['Internal'] = dictionary.get('Internal', None)
                    else:
                        memory_info_dict.update(dictionary)

                # Append data as a dictionary to the list
                all_data_list.append({
                    # General Information
                    "t_Brand": brand_key,
                    "t_Phone": phone_key,

                    # Network Information
                    "t_Network": network_info_dict.get('Technology', None),
                    "t_Network_2G_bands": network_info_dict.get('2G bands', None),
                    "t_Network_3G_bands": network_info_dict.get('3G bands', None),
                    "t_Network_4G_bands": network_info_dict.get('4G bands', None),
                    "t_Network_5G_bands": network_info_dict.get('5G bands', None),
                    "t_Network_Speed": network_info_dict.get('Speed', None),
                    "t_Network_GPRS": network_info_dict.get('GPRS', None),
                    "t_Network_EDGE": network_info_dict.get('EDGE', None),

                    # Launch Information
                    "t_Launch_Announced": launch_info_dict.get('Announced', None),
                    "t_Launch_Status": launch_info_dict.get('Status', None),

                    # Body Information
                    "t_Body_Dimensions": body_info_dict.get('Dimensions', None),
                    "t_Body_Weight": body_info_dict.get('Weight', None),
                    "t_Body_SIM": body_info_dict.get('SIM', None),

                    # Display Information
                    "t_Display_Type": display_info_dict.get('Type', None),
                    "t_Display_Size": display_info_dict.get('Size', None),
                    "t_Display_Resolution": display_info_dict.get('Resolution', None),
                    "t_Display_Protection": display_info_dict.get('Protection', None),

                    # Platform Information
                    "t_Platform_OS": platform_info_dict.get('OS', None),
                    "t_Platform_Chipset": platform_info_dict.get('Chipset', None),
                    "t_Platform_CPU": platform_info_dict.get('CPU', None),
                    "t_Platform_GPU": platform_info_dict.get('GPU', None),

                    # Memory Information
                    "t_Memory_Card_slot": memory_info_dict.get('Card slot', None),
                    "t_Internal_capacity": memory_info_dict.get('t_Internal_capacity', None),
                    "t_Internal": memory_info_dict.get('Internal', None),

                    # Battery Information
                    "t_Battery": battery_info_dict.get('Type', None),

                    # Miscellaneous Information
                    "t_Price": misc_info_dict.get('Price', None),
                    "t_Colors": misc_info_dict.get('Colors', None),

                    # Communication Information
                    "t_Bluetooth": comms_info_dict.get('Bluetooth', None),
                    "t_WLAN": comms_info_dict.get('WLAN', None),

                    # Features
                    "t_Sensors": features_info_dict.get('Sensors', None),

                    # Sound Information
                    "t_Loudspeaker": sound_info_dict.get('Loudspeaker', None),
                    "t_3.5mm_jack": sound_info_dict.get('3.5mm jack', None)
                })

        # Create DataFrame for the current file
        df = pd.DataFrame(all_data_list)
        mobile_dataframes.append(df)
    except ValueError as e:
        print(f"Error reading {json_file}: {e}")


In [140]:
mobile_dataframes[9].columns

Index(['t_Brand', 't_Phone', 't_Network', 't_Network_2G_bands',
       't_Network_3G_bands', 't_Network_4G_bands', 't_Network_5G_bands',
       't_Network_Speed', 't_Network_GPRS', 't_Network_EDGE',
       't_Launch_Announced', 't_Launch_Status', 't_Body_Dimensions',
       't_Body_Weight', 't_Body_SIM', 't_Display_Type', 't_Display_Size',
       't_Display_Resolution', 't_Display_Protection', 't_Platform_OS',
       't_Platform_Chipset', 't_Platform_CPU', 't_Platform_GPU',
       't_Memory_Card_slot', 't_Internal_capacity', 't_Internal', 't_Battery',
       't_Price', 't_Colors', 't_Bluetooth', 't_WLAN', 't_Sensors',
       't_Loudspeaker', 't_3.5mm_jack'],
      dtype='object')

In [130]:
mobile_dataframes[9].head()

,t_Brand,t_Phone,t_Network,t_Network_2G_bands,t_Network_3G_bands,t_Network_4G_bands,t_Network_5G_bands,t_Network_Speed,t_Network_GPRS,t_Network_EDGE,...,t_Internal_capacity,t_Internal,t_Battery,t_Price,t_Colors,t_Bluetooth,t_WLAN,t_Sensors,t_Loudspeaker,t_3.5mm_jack
0,Apple118,iPhone 15 Pro Max,GSM / CDMA / HSPA / EVDO / LTE / 5G,CDMA 800 / 1900,CDMA2000 1xEV-DO,"1, 2, 3, 4, 5, 7, 8, 12, 13, 17, 18, 19, 20, 2...","1, 2, 3, 5, 7, 8, 12, 20, 25, 26, 28, 30, 38, ...","HSPA, LTE-A, 5G, EV-DO Rev.A 3.1 Mbps",None,None,...,"256GB 8GB RAM, 512GB 8GB RAM, 1TB 8GB RAM",NVMe,"Li-Ion 4441 mAh, non-removable","$ 1,095.95 / € 1,299.00 / £ 1,149.00 / ₹ 148,900","Black Titanium, White Titanium, Blue Titanium,...","5.3, A2DP, LE","Wi-Fi 802.11 a/b/g/n/ac/6e, dual-band, hotspot",Ultra Wideband 2 (UWB) supportEmergency SOS vi...,"Yes, with stereo speakers",No
1,Apple118,iPhone 15 Pro,GSM / CDMA / HSPA / EVDO / LTE / 5G,CDMA 800 / 1900,CDMA2000 1xEV-DO,"1, 2, 3, 4, 5, 7, 8, 12, 13, 17, 18, 19, 20, 2...","1, 2, 3, 5, 7, 8, 12, 20, 25, 26, 28, 30, 38, ...","HSPA, LTE-A, 5G, EV-DO Rev.A 3.1 Mbps",None,None,...,"128GB 8GB RAM, 256GB 8GB RAM, 512GB 8GB RAM, 1...",NVMe,"Li-Ion 3274 mAh, non-removable","$ 886.88 / € 1,069.00 / £ 949.00 / ₹ 127,990","Black Titanium, White Titanium, Blue Titanium,...","5.3, A2DP, LE","Wi-Fi 802.11 a/b/g/n/ac/6e, dual-band, hotspot",Ultra Wideband 2 (UWB) supportEmergency SOS vi...,"Yes, with stereo speakers",No
2,Apple118,iPhone 15 Plus,GSM / CDMA / HSPA / EVDO / LTE / 5G,CDMA 800 / 1900,CDMA2000 1xEV-DO,"1, 2, 3, 4, 5, 7, 8, 12, 13, 17, 18, 19, 20, 2...","1, 2, 3, 5, 7, 8, 12, 20, 25, 26, 28, 30, 38, ...","HSPA, LTE-A, 5G, EV-DO Rev.A 3.1 Mbps",None,None,...,"128GB 6GB RAM, 256GB 6GB RAM, 512GB 6GB RAM",NVMe,"Li-Ion 4383 mAh, non-removable","$ 699.44 / € 970.00 / £ 849.00 / ₹ 80,990","Black, Blue, Green, Yellow, Pink","5.3, A2DP, LE","Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot",Ultra Wideband 2 (UWB) supportEmergency SOS vi...,"Yes, with stereo speakers",No
3,Apple118,iPhone 15,GSM / CDMA / HSPA / EVDO / LTE / 5G,CDMA 800 / 1900,CDMA2000 1xEV-DO,"1, 2, 3, 4, 5, 7, 8, 12, 13, 17, 18, 19, 20, 2...","1, 2, 3, 5, 7, 8, 12, 20, 25, 26, 28, 30, 38, ...","HSPA, LTE-A, 5G, EV-DO Rev.A 3.1 Mbps",None,None,...,"128GB 6GB RAM, 256GB 6GB RAM, 512GB 6GB RAM",NVMe,"Li-Ion 3349 mAh, non-removable","$ 699.00 / € 838.00 / £ 749.00 / ₹ 71,990","Black, Blue, Green, Yellow, Pink","5.3, A2DP, LE","Wi-Fi 802.11 a/b/g/n/ac/6, dual-band, hotspot",Ultra Wideband 2 (UWB) supportEmergency SOS vi...,"Yes, with stereo speakers",No
4,Apple118,Watch Ultra 2,GSM / HSPA / LTE,GSM 850 / 900 / 1800 / 1900,"HSDPA 850 / 1700(AWS) / 1900 / 2100 - USA, LAT...","1, 2, 3, 4, 5, 6, 7, 12, 13, 14, 17, 18, 19, 2...",None,"HSPA, LTE",None,None,...,64GB,None,"Li-Ion 564 mAh, non-removable",$ 836.99 / € 64.66,Titanium,"5.3, A2DP, LE","Wi-Fi 802.11 b/g/n, dual-band",Temperature sensing (0.01˚ accuracy)Depth gaug...,"Yes, with dual speakers (86-decibel)",No


In [20]:
dataset = pd.concat(mobile_dataframes, ignore_index=True)

In [16]:
def extract_float(text, pattern):
    """Extracts the first floating number in a text matching a pattern."""
    import re
    if isinstance(text, str):
        match = re.search(pattern, text)
        if match:
            return float(match.group())
    return np.nan

def extract_year(text):
    """Extracts the year from text."""
    if isinstance(text, str):
        return int(''.join(filter(str.isdigit, text[:4]))) if text[:4].isdigit() else np.nan
    return np.nan

def extract_status(status):
    """Extracts status: Coming, Available, Discontinued."""
    if isinstance(status, str):
        status = status.lower()
        if "coming" in status:
            return "Coming"
        elif "available" in status:
            return "Available"
        elif "discontinued" in status:
            return "Discontinued"
    return None

def extract_number(text, unit):
    """Extracts numeric value based on unit."""
    if isinstance(text, str):
        num = ''.join(filter(lambda c: c.isdigit() or c == '.', text))
        if unit in text:
            return float(num)
    return np.nan

def extract_ratio(text):
    """Extracts the ratio in the format number:number before the word 'ratio'."""
    if isinstance(text, str):
        match = re.search(r'(\d+:\d+)\s*ratio', text, re.IGNORECASE)
        if match:
            return match.group(1)
    return np.nan

def extract_ppi(text):
    """Extract pixels-per-inch (ppi) from display resolution."""
    if isinstance(text, str) and "ppi" in text:
        return extract_float(text, r"\d+")
    return np.nan

def extract_weight_regex(text):
    """Extracts weight in grams using regex."""
    if isinstance(text, str):
        match = re.search(r'(\d+\.?\d*)\s*g', text)
        if match:
            return float(match.group(1))
    return np.nan

def extract_battery_capacity_regex(text):
    """Extracts battery capacity in mAh using regex."""
    if isinstance(text, str):
        match = re.search(r'(\d+\.?\d*)\s*mAh', text)
        if match:
            return float(match.group(1))
    return np.nan

def extract_float_fixed(text, pattern):
    """Extracts the numeric value from text using regex, capturing only digits."""
    if isinstance(text, str):
        match = re.search(pattern, text)
        if match:
            return float(match.group(1))  # Extract only the first capturing group (numeric value)
    return np.nan

# Adjust the price extraction function to use the fixed extract_float
def extract_price_enhanced(price_text):
    """Extract price in USD, converting from Euro if needed, and handle various formats."""
    if isinstance(price_text, str):
        # Normalize by removing non-standard spaces
        price_text = price_text.replace('\u2009', '').replace('\xa0', '').replace(',', '')

        # Handle price in USD directly
        if '$' in price_text:
            return extract_float_fixed(price_text, r"\$(\d+\.?\d*)")

        # Handle price in EUR and convert to USD
        if 'EUR' in price_text or '€' in price_text:
            euro_price = extract_float_fixed(price_text, r"(?:About\s+)?(\d+\.?\d*)\s*(?:EUR|€)")
            return euro_price * 1.1 if euro_price else np.nan  # Approx conversion to USD

        # Handle INR or other currencies (optional, can add logic if needed)
        # For now, INR is ignored as not convertible in this scope
    return np.nan

def extract_body_ratio(text):
    """Extracts the body ratio in the format 'number:number' from display resolution text."""
    if isinstance(text, str):
        match = re.search(r'(\d+:\d+)\s*ratio', text, re.IGNORECASE)
        if match:
            return match.group(1)
    return np.nan

def extract_ppi_density(text):
    """Extracts the PPI density from display resolution text."""
    if isinstance(text, str):
        match = re.search(r'~(\d+)\s*ppi', text, re.IGNORECASE)
        if match:
            return int(match.group(1))
    return np.nan

def extract_loudspeaker(text):
    """Determines if a device has a loudspeaker based on the text."""
    if isinstance(text, str):
        if "Yes" in text:
            return True
    return False



In [11]:
# Create the new DataFrame with selected features
processed_df = pd.DataFrame()

# Extract Brand
processed_df['Brand'] = dataset['t_Brand'].str.extract(r'([a-zA-Z]+)')

# Add network band support columns
processed_df['2G'] = dataset['t_Network_2G_bands'].notna()
processed_df['3G'] = dataset['t_Network_3G_bands'].notna()
processed_df['4G'] = dataset['t_Network_4G_bands'].notna()
processed_df['5G'] = dataset['t_Network_5G_bands'].notna()

# Extract Announced year
processed_df['Announced'] = dataset['t_Launch_Announced'].apply(extract_year)

# Extract Status
processed_df['Status'] = dataset['t_Launch_Status'].apply(extract_status)

# Extract dimensions and weight
processed_df['Weight'] = dataset['t_Body_Weight'].apply(extract_weight_regex)
processed_df['Length'] = dataset['t_Body_Dimensions'].str.extract(r'(\d+\.?\d*) x')[0].astype(float)
processed_df['Width'] = dataset['t_Body_Dimensions'].str.extract(r'x (\d+\.?\d*) x')[0].astype(float)
processed_df['Diameter'] = dataset['t_Body_Dimensions'].str.extract(r'x \d+\.?\d* x (\d+\.?\d*)')[0].astype(float)



In [12]:
# Extract Display Type, Size, PPI, Body Ratio
processed_df['Display Type'] = dataset['t_Display_Type'].str.split(',', expand=True)[0].str.strip()
processed_df['Display Size'] = dataset['t_Display_Size'].apply(lambda x: extract_float(x, r"\d+\.?\d*"))
processed_df['PPI'] = dataset['t_Display_Resolution'].apply(extract_ppi_density)
processed_df['Body Ratio'] = dataset['t_Display_Resolution'].apply(extract_body_ratio)

# Extract OS and Battery Capacity
processed_df['OS'] = dataset['t_Platform_OS'].str.extract(r'([a-zA-Z]+[\d\.]*)')
processed_df['Battery Capacity'] = dataset['t_Battery'].apply(extract_battery_capacity_regex)

# Extract Price in USD (convert Euro if needed)
processed_df['Price'] = dataset['t_Price'].apply(extract_price_enhanced)


In [17]:
# Extract CPU cores and clock speed
processed_df['CPU'] = dataset['t_Platform_CPU']

# Extract Ratio, Pixel, WLAN
processed_df['Ratio'] = dataset['t_Display_Resolution'].apply(extract_ratio)
processed_df['Pixel'] = processed_df['PPI']
processed_df['WLAN'] = dataset['t_WLAN'].apply(lambda x: 'Wifi' if isinstance(x, str) and 'Wi-Fi' in x else 'No')

# Extract other features
processed_df['Colors'] = dataset['t_Colors']
processed_df['Sensors'] = dataset['t_Sensors']
processed_df['Bluetooth'] = dataset['t_Bluetooth'].apply(lambda x: extract_float(x, r'\d+\.?\d*'))
processed_df['GPU'] = dataset['t_Platform_GPU'].apply(lambda x: x if pd.notna(x) else 'No')
processed_df['LoudSpeaker'] = dataset['t_Loudspeaker'].apply(extract_loudspeaker)
processed_df['3.5mm Jack'] = dataset['t_3.5mm_jack'].notna()
processed_df['Chipset'] = dataset['t_Platform_Chipset']
processed_df['Network'] = dataset['t_Network']
processed_df['Card Slot'] = dataset['t_Memory_Card_slot'].notna()

# Extract RAM and Storage from Internal Capacity
processed_df['RAM'] = dataset['t_Internal_capacity'].str.extract(r'(\d+GB) RAM')[0]
processed_df['Storage'] = dataset['t_Internal_capacity'].str.extract(r'(\d+GB)')[0]





In [21]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5910 entries, 0 to 5909
Data columns (total 33 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Brand             5910 non-null   object 
 1   2G                5910 non-null   bool   
 2   3G                5910 non-null   bool   
 3   4G                5910 non-null   bool   
 4   5G                5910 non-null   bool   
 5   Announced         5818 non-null   float64
 6   Status            5782 non-null   object 
 7   Weight            5598 non-null   float64
 8   Length            5791 non-null   float64
 9   Width             5786 non-null   float64
 10  Diameter          5786 non-null   float64
 11  Display Type      5910 non-null   object 
 12  Display Size      5530 non-null   float64
 13  PPI               4634 non-null   float64
 14  Body Ratio        4115 non-null   object 
 15  OS                4141 non-null   object 
 16  Battery Capacity  5568 non-null   float64


In [22]:
processed_df.to_csv(folder_path + "mobile_data_regres.csv", index=False)
processed_df.head()

,Brand,2G,3G,4G,5G,Announced,Status,Weight,Length,Width,...,Sensors,Bluetooth,GPU,LoudSpeaker,3.5mm Jack,Chipset,Network,Card Slot,RAM,Storage
0,Asus,True,True,True,True,2024.0,Available,225.0,163.8,76.8,...,None,5.3,Adreno 750,True,True,Qualcomm SM8650-AB Snapdragon 8 Gen 3 (4 nm),GSM / CDMA / HSPA / LTE / 5G,True,16GB,512GB
1,Asus,True,True,True,True,2024.0,Available,225.0,163.8,76.8,...,None,5.3,Adreno 750,True,True,Qualcomm SM8650-AB Snapdragon 8 Gen 3 (4 nm),GSM / CDMA / HSPA / LTE / 5G,True,12GB,256GB
2,Asus,True,True,True,True,2023.0,Available,172.0,146.5,68.1,...,None,5.3,Adreno 740,True,True,Qualcomm SM8550-AB Snapdragon 8 Gen 2 (4 nm),GSM / HSPA / LTE / 5G,True,8GB,128GB
3,Asus,True,True,True,True,2023.0,Available,239.0,173.0,77.0,...,None,5.3,Adreno 740,True,True,Qualcomm SM8550-AB Snapdragon 8 Gen 2 (4 nm),GSM / CDMA / HSPA / LTE / 5G,True,16GB,512GB
4,Asus,True,True,True,True,2023.0,Available,239.0,173.0,77.0,...,None,5.3,Adreno 740,True,True,Qualcomm SM8550-AB Snapdragon 8 Gen 2 (4 nm),GSM / CDMA / HSPA / LTE / 5G,True,12GB,256GB
